In [ ]:
from analysis import collectConnections, print_with_stats
import pandas as pd
import re
import json

In [ ]:

to_namedtuple = lambda x: list(x.itertuples())
actions = to_namedtuple(pd.read_csv('../data/all-history-actions.csv'))
actions_df = pd.read_csv('../data/all-history-actions.csv')
findBillNum = re.compile('([HS]\d{1,5})')

In [ ]:
# TODO: parse action types: "referred to", "hearing scheduled", "reported from"
# TODO: create parse function that converts a raw action string to an array of dicts. Each dict has a 'type' field that identifies its type, and additional attribute fields

referred = re.compile(
    "(?P<type>referred to the committee) on (?P<committee_name>.*)", flags=re.IGNORECASE
)

hearing_scheduled = re.compile(
    "(?P<type>hearing scheduled) for (?P<date>.*) from (?P<time>.*) in (?P<forum>.*)", flags = re.IGNORECASE
)

reported_from = re.compile(
    "(?P<type>reported) from (?P<committee_name>.*)", flags= re.IGNORECASE
)

reading = re.compile(
    "(?P<type>read) (?P<reading>(second|third|second and ordered to a third))", flags= re.IGNORECASE
)

enacted = re.compile(
    "(?P<type>enacted) and (?P<info>.*)", flags= re.IGNORECASE
)

enacted_votes = re.compile(
    "(?P<type>enacted) - (?P<info>.*)", flags= re.IGNORECASE
)


action = actions[0].action
r = referred.match(action)

# actions[0] is a namedtuple. You can convert it to a dict with actions[0]._asdict()
# display(actions[0])
# print(action, r.groupdict())
# print(actions[0]._asdict().keys())

# def parse(action):
#     return [{"type": "reported"}, {"type": "referred", "committee_name": "Technology"}]

In [ ]:

action_types = [reported_from, referred]

#creates list of actions from dictionaries
def extract_actions(actions):
    action_dict = action_to_dict(actions)
    extracted_actions = [action['action'] for action in action_dict]
    return extracted_actions


#converts list of named tuples to a list of dictionaries
def action_to_dict(actions):
    action_dict = [action._asdict() for action in actions]
    return action_dict


#creates list of dictionaries of matching action types
def parse_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_actions.append(selected_action.groupdict())
    return selected_actions

#creates list of enacted actions
def append_enacted_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'info': selected_action_dict['info']}
                    selected_actions.append(selected_dict)
    return selected_actions

#creates list of reported and referred actions
def append_committee_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'committee_name': selected_action_dict['committee_name']}
                    selected_actions.append(selected_dict)
    return selected_actions

#creates list of hearing scheduled actions
def append_hearing_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'scheduled_date': selected_action_dict['date'], 'time': selected_action_dict['time'], 'forum': selected_action_dict['forum']}
                    selected_actions.append(selected_dict)
    return selected_actions


In [ ]:
action_dict = parse_actions(action_types, action_to_dict(actions))

action_dict


In [ ]:
appended_action_types = [enacted, enacted_votes]

append_enacted_actions(appended_action_types, action_to_dict(actions))

In [ ]:
#exports parsed actions to a JSON file
def write_json(dictionary, str):
    json_object = json.dumps(dictionary, indent=4)
    with open(f"../data/{str}.json", "w") as outfile:
        outfile.write(json_object)

In [ ]:
write_json(append_enacted_actions(appended_action_types, action_to_dict(actions)), 'enacted')

In [ ]:
action_types = [reported_from, referred]
write_json(append_committee_actions(action_types, action_to_dict(actions)), 'reported-referred')

In [ ]:
action_types = [hearing_scheduled]
write_json(append_hearing_actions(action_types, action_to_dict(actions)), 'hearing_scheduled')